In [1]:
from transformers import AutoTokenizer, AutoModel
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer
import torch
from torch import nn

import pandas as pd

In [2]:
model = AutoModel.from_pretrained("csebuetnlp/banglabert")
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert")

In [3]:
original_sentence = "আমি কৃতজ্ঞ কারণ আপনি আমার জন্য অনেক কিছু"
fake_sentence = "আপনি হতাশ কারণ  আমার জন্য অনেক কিছু করেছেন।"
fake_sentence = normalize(fake_sentence) # this normalization step is required before tokenizing the text
original_sentence = normalize(original_sentence)

In [4]:
fake_tokens = tokenizer.tokenize(fake_sentence)
fake_inputs = tokenizer.encode_plus(fake_sentence, return_tensors="pt")
# fake_inputs

original_inputs = tokenizer.encode_plus(original_sentence, return_tensors="pt")

In [5]:
discriminator_outputs = model(fake_inputs['input_ids'],attention_mask= fake_inputs['attention_mask'])#.logits
discriminator_outputs2 = model(original_inputs['input_ids'],attention_mask= original_inputs['attention_mask'])#.logits
# predictions = torch.round((torch.sign(discriminator_outputs) + 1) / 2)

In [6]:
input1 = discriminator_outputs['last_hidden_state'][:,0,:]
input2 = discriminator_outputs2['last_hidden_state'][:,0,:]

In [23]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [8]:
output = cos(input1, input2)
output

tensor([0.8526], grad_fn=<SumBackward1>)

In [9]:
df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)
df_test = pd.read_csv("dataset/bornon_test_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)

/tmp/ipykernel_31490/2331687903.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_train = pd.read_csv("dataset/bornon_train_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)
/tmp/ipykernel_31490/2331687903.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df_test = pd.read_csv("dataset/bornon_test_token.txt", delimiter="#0\s+", names=["image_name", "caption"], header=None)


In [10]:
def consolidate(gdf):
    captions = gdf['caption'].to_list()
    return captions

df1 = df_train.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})
df2 = df_test.groupby("image_name").apply(consolidate, include_groups=False).reset_index().rename(columns={0: 'captions'})

In [11]:
df1.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)
df2.sort_values(by='image_name', key=lambda x: pd.to_numeric(x.str.rstrip('.jpg'), errors='coerce'), inplace=True)

df = pd.concat([df1, df2], axis=0)
df = df.reset_index()

In [24]:
import itertools

In [131]:
def top_3_captions(captions: list):
    """
    top 3 being the max length string, minimum length string and the string which has maximum cosine similarity index with 
    both of the max and min length string

    Args:
        captions (list): list of 5 captions from the bornon dataset for each image

    Returns:
        _type_: list of top 3 captions
    """
    cap_discriminator_outputs = []
    cap_inputs = []
    cap_final = []
    
    # cap_final.append(tokenizer.encode_plus(normalize(max(captions, key=len)), return_tensors="pt"))
    # cap_final.append(tokenizer.encode_plus(normalize(min(captions, key=len)), return_tensors="pt"))
    cap_final.append(normalize(max(captions, key=len)))  
    cap_final.append(normalize(min(captions, key=len)))  
    
    for i in range(len(captions)):
        normalised = normalize(captions[i])
        cap_inputs.append(tokenizer.encode_plus(normalised, return_tensors="pt"))
    
    for i in range(len(cap_inputs)):
        cap_discriminator_outputs.append(model(cap_inputs[i]['input_ids'],attention_mask= cap_inputs[i]['attention_mask'])['last_hidden_state'][:,0,:])
    
    max_len_str = tokenizer.encode_plus(cap_final[0], return_tensors="pt")
    min_len_str = tokenizer.encode_plus(cap_final[1], return_tensors="pt")
    
    max_len_str = model(max_len_str['input_ids'],attention_mask= max_len_str['attention_mask'])['last_hidden_state'][:,0,:]
    min_len_str = model(min_len_str['input_ids'],attention_mask= min_len_str['attention_mask'])['last_hidden_state'][:,0,:]
    
    
    # print(max_len_str.shape)
    
    max_val = -1
    idx = 0
    for i in range(len(cap_discriminator_outputs)):
        # print(i)
        if cos(max_len_str, cap_discriminator_outputs[i]) > 0.95 or cos(min_len_str, cap_discriminator_outputs[i]) > 0.95:
            # print(captions[i], cap_final[0], cap_final[1])
            pass
            
        else:
            val1 = cos(max_len_str, cap_discriminator_outputs[i])
            val2 = cos(min_len_str, cap_discriminator_outputs[i])
            
            # print(captions[i])
            # print(captions[i], cap_final[0], cap_final[1])
            
            if val1+val2 > max_val:
                max_val = val1+val2
                idx = i
                
    cap_final.append(normalize(captions[idx]))
    
    return (cap_final)
    

In [128]:
df['top_captions'] = None
from tqdm import notebook as nb

In [129]:
for idx, row in nb.tqdm(df.iterrows()):
    
    outs = top_3_captions(captions=row['captions'])
    # df.loc[idx, 'top_captions'] = ',\n '.join(map(str, outs))
    
    df.loc[df['image_name'] == row['image_name'], 'top_captions'] = ',\n '.join(map(str, outs))
    

0it [00:00, ?it/s]

In [120]:
df.head()

,index,image_name,captions,top_captions
0,0,1.jpg,"[একটি পার্কে একটি লেক আছে ।, লেকের উপর একটি ব্...","গাছপালায় ঘেরা এই পার্কটি খুবই মনমুগ্ধকর ।,\n ..."
1,934,2.jpg,"[একজন মানুষ মাথায় খড় নিয়ে হেটে যাচ্ছে ।, এক...",লুঙ্গি পড়া একজন মানুষ খড়ের উপর হেটে যাচ্ছে ।...
2,1885,3.jpg,"[টেবিলের উপর একটি চশমা আছে ।, টেবিলের উপর একটি...","একটি রুমে একটি খাট এবং একটি টেবিল আছে,টেবিলের ..."
3,2829,4.jpg,"[একটি শিশু দেখা যাচ্ছে ।, একটি শিশু গ্রামের ছো...",একটি শিশু হাটার সময় তার কিছুটা ছায়া দেখা যাচ...
4,3013,5.jpg,"[একটি চায়ের কাপ দেখা যাচ্ছে ।, চায়ের কাপে উপ...","আলোর কারনে চায়ের কাপের ছায়া দেখা যাচ্ছে ।,\n..."


In [130]:
df.to_csv("top_captioned.csv", index=False)